# Patch-clamp: passive membrane properties

In this tutorial, I explain how to calculate the basic passive membrane properties and voltage from current clamp recordings using custom functions. To read the full tutorial, please see [Patch-clamp data analysis in Python: passive membrane propertiesPatch-clamp data analysis in Python: passive membrane properties](https://spikesandbursts.wordpress.com/2022/05/13/patch-clamp-data-analysis-python-passive-membrane-properties/) of the [Spikes and Bursts](https://spikesandbursts.wordpress.com/) blog.


# Import the libraries

In [ ]:
import os
import pyabf
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Define the paths to folders and files

In [ ]:
notebook_name = 'passive_membrane_properties'

# Data path to 'Data_example' folders. Change accordingly to your data structure.
data_path = os.path.dirname(os.getcwd())  # Moves one level up from the current directory

# Change the folder names accordingly
paths = {'data':  f'{data_path}/Data',
         'processed_data': f'{data_path}/Processed_data/{notebook_name}',
         'analysis': f'{data_path}/Analysis/{notebook_name}'}

# Make folders if they do not exist yet
for path in paths.values():
    os.makedirs(path, exist_ok=True)

# Load the data

The example data for this notebook is the file **pfc_pyr_passive.abf**.

In [ ]:
# ABF file/s
filename = "pfc_pyr_passive"

data_path = f"{paths['data']}/{filename}.abf" 
abf = pyabf.ABF(data_path)
print(abf)

# Quick plot to see the trace/s
plt.figure(figsize=(8,4))

for sweepNumber in abf.sweepList:
    abf.setSweep(sweepNumber)
    plt.plot(abf.sweepX, abf.sweepY)
    plt.ylabel(abf.sweepLabelY)
    plt.xlabel(abf.sweepLabelX)

plt.show()

# Select the sweep and/or channel
# abf.setSweep(10)  # Sweep
# abf.setSweep(sweepNumber=0, channel=0)  # Sweep and channel

# Passive membrane properties

## Function

I have updated the function (April 2023) to allow the definition of each trace separately. This means that you can use a filtered voltage trace if needed or different data than .abf files.

**Note**: instead of using `global`, you can use `return` an specific variable: e.g. `c`, and then call the results like `features`, `c`* Note: instead of using `global`, you can use `return` an specific variable: e.g. `c`, and then call the results like `features`, `c`

**Documentation**:
- [Scipy curve fitting]( https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)

In [ ]:
def passive_memb_properties(voltage_signal, current_signal, time_signal, fs, 
                            cursor3_ms, cursor4_ms, cursor5_ms, 
                            a_initial, tau_initial, c_initial, 
                            cursor1_ms=5, cursor2_ms=50):
    """
    Inputs: 
    voltage_signal: Voltage trace, raw or pre-processed
    current_signal: Current trace
    time_signal: Time trace
    fs: Sampling frequency
    cursor3_ms: Time cursor position in ms set at the peak of the capacitive transient.
    cursor4_ms: Time cursor position in ms set at the end of the capacitive transient.
    cursor5_ms: Time cursor position in ms set at the end of the voltage step.
    a_initial: Initial estimate for exponential fitting.
    tau_initial: Initial estimate for exponential fitting.
    c_initial: Initial estimate for exponential fitting.
    cursor1_ms: Optional, default value is 5
    cursor2_ms: Optional, default value is 50
    
    Returns:
    rmp: Resting membrane potential
    current_steps: Current step values
    voltage_delta: Delta voltage response to current step between cursor 3 and 4
    steady_voltage: Steady voltage response between cursor 4 and 5
    ir: Input resistance in MOhm
    tau: Decay time constant in ms
    r_squared: Quality of the exponential fitting
    capacitance: Capacitance of the passive membrane in pF
    """ 

    # Position of cursors in time bins (depends on sampling frequency)
    global cursor3, cursor4, cursor5
    cursor1 = int(cursor1_ms * (fs/1000))
    cursor2 = int(cursor2_ms * (fs/1000))
    cursor3 = int(cursor3_ms * (fs/1000))
    cursor4 = int(cursor4_ms * (fs/1000))
    cursor5 = int(cursor5_ms * (fs/1000))

    # steady voltage response
    steady_voltage = np.average(voltage_signal[cursor4:cursor5])
    
    # Resting membrane potential
    rmp = np.average(voltage_signal[cursor1:cursor2])

    # Get the current step values
    current_steps = np.average(current_signal[cursor4:cursor5])

    # Delta voltage response to current step
    voltage_delta = np.average(voltage_signal[cursor4:cursor5]) - rmp

    # Input resistance
    ir = abs(voltage_delta/current_steps)*1000

    # Calculate tau from decay time constant
    decay_voltage = voltage_signal[cursor3:cursor4]
    decay_time = (time_signal[cursor3:cursor4])*1000
    decay_voltage = decay_voltage[~np.isnan(decay_voltage)]
    decay_time = decay_time[~np.isnan(decay_time)]

    # Initial values (a, tau, c) for estimating the fitting
    a_initial = a_initial      # Initial value  
    tau_initial = tau_initial  # Estimated tau
    c_initial = c_initial      # Baseline value
    
    # popt: optimal values for the parameters
    # pcov: estimated covariance of popt
    global a, tau, c
    popt, pcov = curve_fit(lambda t, a, tau, c: a * np.exp(-t/tau) + c, 
                           xdata = decay_time,     # x-data
                           ydata = decay_voltage,  # y-data
                           p0 = (a_initial, tau_initial, c_initial))
    a = popt[0]
    tau = popt[1]
    c = popt[2]

    # Quality of the exponential fitting
    # RSS: Residual sum of squares
    RSS = np.square(decay_voltage - (a * np.exp(-decay_time/tau) + c))
    # TSS: total sum of squares
    TSS = np.square(decay_voltage - np.mean(decay_voltage))
    r_squared = 1 - np.sum(RSS) / np.sum(TSS)
    
    # capacitance
    capacitance = (tau/ir) * 1000
       
    return {'rmp': rmp, 
            'current_steps': current_steps, 
            'voltage_delta': voltage_delta, 
            'steady_voltage': steady_voltage,
            'ir': ir, 
            'tau': tau, 
            'r_squared': r_squared, 
            'capacitance': capacitance}

## Set the analysis parameters

In [ ]:
# Sampling frequency
fs = int(abf.dataPointsPerMs * 1000)

# Define channels of each signal
global voltage_channel, current_channel
voltage_channel = 0
current_channel = 0

# Create a list the position of cursors in ms
cursors_ms = [5, 100, 130, 280, 600]

# Initial estimation for exponential fitting (optional
a_initial = 20   # Vpeak - Vsteady
tau_initial = 20  # Estimated tau
c_initial = -70    # Steady voltage

# Sweep with zero current
sweep_zero_current = 3

# Quick plot to visualize the plot and the cursors position
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
for sweep in abf.sweepList:
    abf.setSweep(sweep, channel=voltage_channel)
    ax1.plot(abf.sweepX*1000, abf.sweepY)
        
    abf.setSweep(sweep, channel=current_channel)
    ax2.plot(abf.sweepX*1000, abf.sweepC)

# Labels
ax1.set_ylabel(abf.sweepLabelY)
ax2.set_ylabel(abf.sweepLabelC)
ax2.set_xlabel('Time (ms)')
    
# Plot cursors dotted lines 1 to 5 and ticks
for i, cursor in enumerate(cursors_ms):
    ax1.axvline(cursor, linestyle="dotted", color='gray', label=str(i+1))
    
ax1top= ax1.secondary_xaxis('top')
ax1top.set_xticks(cursors_ms)
ax1top.set_xticklabels(['1', '2', '3', '4', '5']) 

# Save the plot
fig.savefig(f"{paths['analysis']}/{filename}.png", dpi=300)  # or svg, tif, etc.
 
plt.show() 

## Table with the results

In [ ]:
# Create table with results
table = pd.DataFrame()

# Number of decimals in the table
pd.options.display.float_format = '{:.4f}'.format

# Calculate the results for each sweep
for sweep in abf.sweepList:  # [ : ] to select a range of traces

    # Define channels and signals for the function
    abf.setSweep(sweep, channel=voltage_channel)
    voltage_signal = abf.sweepY
    
    abf.setSweep(sweep, channel=current_channel)
    current_signal = abf.sweepC
    
    # Call the function 'passive_memb_properties'
    features = passive_memb_properties(voltage_signal=voltage_signal,
                                       current_signal=current_signal,
                                       time_signal=abf.sweepX,
                                       fs=fs,
                                       cursor3_ms=cursors_ms[2],
                                       cursor4_ms=cursors_ms[3],
                                       cursor5_ms=cursors_ms[4],
                                       a_initial=a_initial,
                                       tau_initial=tau_initial, 
                                       c_initial=c_initial) 
    
    # Fill the table with the results from the function
    length = len(table)
    table.loc[length, 'rmp_mV'] = features['rmp']
    table.loc[length, 'current_steps_pA'] = features['current_steps']
    table.loc[length, 'V_delta_mV'] = features['voltage_delta']
    table.loc[length, 'V_steady_mV'] = features['steady_voltage']
    if table.loc[length, 'current_steps_pA'] != 0: 
        table.loc[length, 'Rin_MOhm'] = features ['ir'] 
        table.loc[length, 'tau_ms'] = features['tau']
        table.loc[length, 'tau_R squared'] = features['r_squared']
        table.loc[length, 'capacitance_pF'] = features['capacitance']

# Save the table
table.to_csv(f"{paths['analysis']}/{filename}_passive_memb_properties.csv", index=False)

table

## Plot with fitting results and I-V curve

In [ ]:
# Plotting (optional): general settings
fig = plt.figure(figsize=(18, 5))  # Figure size
plt.rcParams.update({'font.size': 12})  # Font size        

# Graph 1: Raw traces
ax1 = fig.add_subplot(131)
ax1.set_title('Graph 1: Raw traces')
for sweep in abf.sweepList:
    abf.setSweep(sweep, channel=voltage_channel)
    voltage = abf.sweepY
    time = abf.sweepX*1000  # time in ms
    ax1.plot(time, voltage)

# Labels for cursors 1 to 5
ax1top= ax1.secondary_xaxis('top')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Voltage (mV)')
ax1.set_xlim(0, 1000)   # limit the x range to display (in ms)

# Loop to plot cursor dotted lines 1 to 5
for i, cursor in enumerate(cursors_ms):
    ax1.axvline(cursor, linestyle="dotted", color='gray', label=str(i+1))

# Cursors' tick and labels
ax1top.set_xticks(cursors_ms)
ax1top.set_xticklabels(['1', '2', '3', '4', '5']) 

# Graph 2: Voltage decay and tau fitting
ax2 = fig.add_subplot(132)
ax2.set_title('Graph 2: Tau fitting')

for sweep in abf.sweepList:
    
    # Define channels and signals for the function
    abf.setSweep(sweep, channel=voltage_channel)
    voltage_signal = abf.sweepY

    abf.setSweep(sweep, channel=current_channel)
    current_signal = abf.sweepY 
    
    features = passive_memb_properties(voltage_signal=voltage_signal,
                                       current_signal=current_signal,
                                       time_signal=abf.sweepX,
                                       fs=fs,
                                       cursor3_ms=cursors_ms[2],
                                       cursor4_ms=cursors_ms[3],
                                       cursor5_ms=cursors_ms[4],
                                       a_initial=a_initial,
                                       tau_initial=tau_initial, 
                                       c_initial=c_initial) 
    
    # Voltage decay and tau fitting
    decay_voltage = voltage_signal[cursor3:cursor4]
    decay_time = abf.sweepX[cursor3:cursor4] * 1000
    v_fitted = a * np.exp(-decay_time / tau) + c
    ax2.plot(decay_time, decay_voltage, color='gray', alpha=0.5)
    ax2.plot(decay_time, v_fitted)
    ax2.set_xlabel('Time window (ms)')
    ax2.set_ylabel('Voltage (mV)')

# Graph 3: I-V fitting to calculate input resistance
ax3 = fig.add_subplot(133)
ax3.set_title('Graph 3: I-V')

current = table.loc[:, 'current_steps_pA']
voltage = table.loc[:, 'V_steady_mV']
voltage_list = voltage.values.tolist()
current_list = current.values.tolist()
m, b = np.polyfit(current_list, voltage_list, 1)
line = np.polyval([m, b], current)

ax3.plot(current, line, color='r')
ax3.scatter(current, voltage)
ax3.set_xlabel('Current steps (pA)')
ax3.set_ylabel('Voltage (mV)')

fig.tight_layout()  # Fit subplots within your figure cleanly

# Save the plot
fig.savefig(f"{paths['analysis']}/{filename}_passive_membr_properties.png", dpi=300)  # or svg, tif, etc.

# Print the input resistance from the I-V fitting
print('Input resitance (MOhm) =', m*1000)

# Print the mean capacitance from all traces except the zero current sweep
print('Capacitance (pF) =', table.drop(sweep_zero_current)['capacitance_pF'].mean())

plt.show()

## Example with CSV file

The function also works with csv files. For this example I use the file **pfc_pyr_passive.csv**.

In [ ]:
# Load the data
filename = "pfc_pyr_passive"

data_path = f"{paths['data']}/{filename}.csv" 
recording_csv = pd.read_csv(data_path, keep_default_na=True)
recording_csv

In [ ]:
# Define the analysis parameters
cursors_ms = [5, 100, 130, 280, 600]

# Initial estimation for exponential fitting
a_initial = 20   # RMP
tau_initial = 20  # Estimated tau
c_initial = -70   # Steady voltage

In [ ]:
# Create table with results
table = pd.DataFrame()
# Number of decimals in the table
pd.options.display.float_format = '{:.4f}'.format

# Identify voltage and current columns 
voltage_cols = [col for col in recording_csv.columns if col.startswith('voltage')]
current_cols = [col for col in recording_csv.columns if col.startswith('current')]

# Loop through the voltage columns
for i in range(len(voltage_cols)):
    col_voltage = voltage_cols[i]
    col_current = current_cols[i]
    
    features = passive_memb_properties(
        voltage_signal=recording_csv[col_voltage].values,
        current_signal=recording_csv[col_current].values,
        time_signal=recording_csv.iloc[:, 0].values/1000,
        fs=20000,
        cursor3_ms=cursors_ms[2],
        cursor4_ms=cursors_ms[3],
        cursor5_ms=cursors_ms[4],
        a_initial=a_initial,
        tau_initial=tau_initial, 
        c_initial=c_initial
    )
    
    length = len(table)
    table.loc[length, 'rmp_mV'] = features['rmp']
    current = recording_csv[col_current].values
    current_baseline = np.average(current[cursors_ms[3]:cursors_ms[4]])
    
    table.loc[length, 'current_steps_pA'] = features['current_steps'] - current_baseline
    table.loc[length, 'V_delta_mV'] = features['voltage_delta']
    table.loc[length, 'V_steady_mV'] = features['steady_voltage']
    if table.loc[length, 'current_steps_pA'] != 0: 
        table.loc[length, 'Rin_MOhm'] = (features['voltage_delta']/table.loc[length, 'current_steps_pA'])*1000
        table.loc[length, 'tau_ms'] = features['tau'] 
        table.loc[length, 'tau_R squared'] = features['r_squared']
        table.loc[length, 'capacitance_pF'] = features['capacitance']

# Plotting (optional): general settings
fig = plt.figure(figsize=(18, 5))  # Figure size
plt.rcParams.update({'font.size': 12})  # Font size        

# Graph 1: Raw traces
ax1 = fig.add_subplot(131)
ax1.set_title('Graph 1: Raw traces')
for col_voltage in voltage_cols:
    voltage = recording_csv[col_voltage].values
    time = recording_csv.iloc[:, 0].values
    ax1.plot(time, voltage)

# Labels for cursors 1 to 5
ax1top = ax1.secondary_xaxis('top')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Voltage (mV)')
# Limit the x range to display in the x-axis
ax1.set_xlim(0, 1000)   

# Loop to plot cursor dotted lines 1 to 5
for i, cursor in enumerate(cursors_ms):
    ax1.axvline(cursor, linestyle="dotted", color='gray', label=str(i+1))
    
# Cursors' tick and labels
ax1top.set_xticks(cursors_ms)
ax1top.set_xticklabels(['1', '2', '3', '4', '5']) 

# Graph 2: Voltage decay and tau fitting
ax2 = fig.add_subplot(132)
ax2.set_title('Graph 2: Tau fitting')

for i in range(len(voltage_cols)):
    col_voltage = voltage_cols[i]
    col_current = current_cols[i]
    
    passive_memb_properties(
        voltage_signal=recording_csv[col_voltage].values,
        current_signal=recording_csv[col_current].values,
        time_signal=recording_csv.iloc[:, 0].values/1000,
        fs=20000,
        cursor3_ms=cursors_ms[2],
        cursor4_ms=cursors_ms[3],
        cursor5_ms=cursors_ms[4],
        a_initial=a_initial,
        tau_initial=tau_initial,
        c_initial=c_initial
    )

    voltage = recording_csv[col_voltage].values
    time = recording_csv.iloc[:, 0].values
    decay_voltage = voltage[cursor3:cursor4]
    decay_time = time[cursor3:cursor4]
    v_fitted = a * np.exp(-decay_time/tau) + c
                                          
    ax2.plot(decay_time, decay_voltage, color='gray', alpha=0.5)
    ax2.plot(decay_time, v_fitted)
    ax2.set_xlabel('Time window (ms)')
    ax2.set_ylabel('Voltage (mV)')

# Graph 3: I-V fitting to calculate input resistance
ax3 = fig.add_subplot(133)
ax3.set_title('Graph 3: I-V')

current = table.loc[:, 'current_steps_pA']
voltage = table.loc[:, 'V_steady_mV']
voltage_list = voltage.values.tolist()
current_list = current.values.tolist()
m, b = np.polyfit(current_list, voltage_list, 1)
line = np.polyval([m, b], current)

ax3.plot(current, line, color='r')
ax3.scatter(current, voltage)
ax3.set_xlabel('Current steps (pA)')
ax3.set_ylabel('Voltage (mV)')

fig.tight_layout()  # Fit plots within your figure cleanly

# Print the input resistance from the I-V fitting
print('Input resitance (MOhm) =', m*1000)

# Save plot and table
table.to_csv(f"{paths['analysis']}/{filename}_csv_passive_memb_properties.csv", index=False)
fig.savefig(f"{paths['analysis']}/{filename}_csv_passive_membr_properties.png", dpi=300)  # or svg, tif, etc.

plt.show()
table